In [17]:
pip install pynput

Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install pyautogui

In [19]:
import cv2
import mediapipe as mp
import pyautogui
import random

#to control the screen and be able to press buttons virtualy 
from pynput.mouse import Button, Controller
mouse = Controller()


screen_width, screen_height = pyautogui.size()

mpHands = mp.solutions.hands

## We detect the finger parts 
### using the following numbers in the the picture below 

![alt text](hand_landmarks.png)

### we get the distance and angle between points to detect movement of the hand 

In [20]:
import numpy as np

def get_angle(a, b, c):
    #we see the angle between ab and bc to detect if finger is bent or straight 
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(np.degrees(radians))
    return angle


def get_distance(landmark_ist):
    if len(landmark_ist) < 2:
        return
    (x1, y1), (x2, y2) = landmark_ist[0], landmark_ist[1]
    L = np.hypot(x2 - x1, y2 - y1)
    return np.interp(L, [0, 1], [0, 1000])

In [21]:

def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        hand_landmarks = processed.multi_hand_landmarks[0]  # Assuming only one hand is detected
        index_finger_tip = hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP] #detect the tip of the index finger 
        return index_finger_tip
    return None, None

In [22]:
def move_mouse(index_finger_tip):
    # take co-ordinates of the index finger's tips and make it match the cordinates on the screen 
    if index_finger_tip is not None:
        x = int(index_finger_tip.x * screen_width)
        y = int(index_finger_tip.y / 2 * screen_height)
        pyautogui.moveTo(x, y)


def is_left_click(landmark_list, thumb_index_dist): #find if the index is bent and middle finger and thumb are straight 
    return (
            get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and
            get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) > 90 and
            thumb_index_dist > 50
    )


def is_right_click(landmark_list, thumb_index_dist): #find if the middle finger is bent and index finger and thumb are straight
    return (
            get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50 and
            get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) > 90  and
            thumb_index_dist > 50
    )


def is_double_click(landmark_list, thumb_index_dist): #both index and middle fingers are bent and thumb is straight 
    return (
            get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and
            get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50 and
            thumb_index_dist > 50
    )


<ul>
    <li>if thumb is bent and index and middle finger are straight the mouse follows the cordinates of the index's tip cordinates 
    <li>if middle finger and thumb is straight and index finger is bent then left click 
    <li>if index finger and thumb is straight and middle finger is bent then right click 
    <li>if both middle finger and index finger are bent then double click 
<ul\>

In [23]:
def detect_gesture(frame, landmark_list, processed):
    if len(landmark_list) >= 21:

        index_finger_tip = find_finger_tip(processed)
        thumb_index_dist = get_distance([landmark_list[4], landmark_list[5]])

        if get_distance([landmark_list[4], landmark_list[5]]) < 60  and get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) > 90:
            move_mouse(index_finger_tip)
        elif is_left_click(landmark_list,  thumb_index_dist):
            mouse.press(Button.left)
            mouse.release(Button.left)
            cv2.putText(frame, "Left Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        elif is_right_click(landmark_list, thumb_index_dist):
            mouse.press(Button.right)
            mouse.release(Button.right)
            cv2.putText(frame, "Right Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        elif is_double_click(landmark_list, thumb_index_dist):
            pyautogui.doubleClick()
            cv2.putText(frame, "Double Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

In [24]:
import cv2
import mediapipe as mp
import mediapipe as mp
import pyautogui
import random
from pynput.mouse import Button, Controller
mouse = Controller()

In [25]:
screen_width, screen_height = pyautogui.size() # detecting the hand 
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands=1
)


In [26]:

def main():
    draw = mp.solutions.drawing_utils
    cap = cv2.VideoCapture(0)
    draw = mp.solutions.drawing_utils
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.flip(frame, 1)
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            processed = hands.process(frameRGB)

            landmark_list = []
            if processed.multi_hand_landmarks:
                hand_landmarks = processed.multi_hand_landmarks[0]  # Assuming only one hand is detected
                draw.draw_landmarks(frame, hand_landmarks, mpHands.HAND_CONNECTIONS)
                for lm in hand_landmarks.landmark:
                    landmark_list.append((lm.x, lm.y))

            detect_gesture(frame, landmark_list, processed)

            cv2.imshow('Frame',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):  #press "q" to close the camera window 
                break
    finally:
         cap.release()
         cv2.destroyAllWindows()

 
if __name__ == '__main__':
    main()          